<a href="https://colab.research.google.com/github/mc2engel/noneq_opt/blob/main/Ising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions ##

Since `noneq_opt` is currently private, you need to create a [Personal Access Token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token). Once you have the token, enter it below, and you should be able to run the cell to pip install the package.

To run with a GPU, go to `Runtime > Change runtime type` and choose `GPU`.

<!---TODO: add TPU instructions and code.--->

In [ ]:
token=''
!pip install git+https://$token@github.com/mc2engel/noneq_opt.git

In [48]:
import tqdm

from google.colab import files

import jax
import jax.numpy as jnp

from noneq_opt import ising

import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
size = 2048  #@param
temperature = .5  #@param
field = 0.  #@param

seed = 0  #@param

time_steps = 1000 #@param
save_every = 20  #@param


split, key = jax.random.split(jax.random.PRNGKey(1))

schedule = ising.IsingParameters(jnp.log(temperature) * jnp.ones(save_every),
                                 field * jnp.ones(save_every))
spins = ising.random_spins([size, size], .5, split)

frames = [spins]

for _ in tqdm.trange(time_steps // save_every):
  split, key = jax.random.split(key)
  final_state, summary = ising.simulate_ising(schedule, spins, split)
  spins = final_state.spins
  frames.append(spins)

In [ ]:
# Create an animation of the Ising simulation.

fig = plt.figure(figsize=[12, 12])
ax = plt.axes()

def frame(j):
  ax.clear()
  plot = ax.imshow(frames[j])
  return plot,

print('Building animation...')
anim = animation.FuncAnimation(
    fig, frame, blit=True, frames=time_steps // save_every)
plt.close(fig)
anim

In [ ]:
# Download the animation

path = '/tmp/ising.mp4'
anim.save(path)
files.download(path)